In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import re
import nltk
import spacy
from collections import Counter
from textblob import TextBlob
import emoji
from unidecode import unidecode
import unicodedata
%matplotlib inline

In [2]:
data_path = '../data/practica_2/marcelo1.txt'

## Leer archivo de texto de conversación

In [3]:
def lectura(text):
    """
    Esta función recibe como parámetro un txt de whatsapp y regresa un dataframde de pandas
    v1.0.0
    """
    fecha = []
    hora = []
    usuario = []
    mensaje = []
    with open(text) as f:
        content = f.readlines()
    for row in content:
        h = row.split(']')
        fecha.append(h[0].replace('[', '').lstrip().strip().strip("\u200e"))
        # The user name is between the last ']' and the first ':'
        # El nombre de usuario está entre el último ] y el primer :
        usuario.append(row[row.find('] ') + len('] '):row.find(': ')])
        # The message is after the inmediate ':' symbol after user name
        # El mensaje está inmediatamnete despés del : después del usuario
        mensaje.append(row[row.find(': ') + 1 :].replace('\n', ' ').lstrip().strip().strip("\u200e"))
    df = pd.DataFrame(zip(fecha, usuario, mensaje), columns = ['t_fecha', 'v_usuario', 't_mensaje'])
    f.close()
    return df

In [4]:
df = lectura(data_path)

In [5]:
df

,t_fecha,v_usuario,t_mensaje
0,"26/10/20, 0:13:27",Marcelo,Los mensajes y las llamadas están cifrados de ...
1,"26/10/20, 0:13:27",Marcelo,https://fb.watch/1m8FhbxovL/
2,"26/10/20, 0:13:34",Marcelo,Buenas
3,"26/10/20, 6:59:08",Unidαd Absolutα,Wey
4,"26/10/20, 6:59:17",Unidαd Absolutα,Está bien vergas este horario
...,...,...,...
61436,"19/05/22, 12:20:30",Unidαd Absolutα,Va a salir con sida wey
61437,"19/05/22, 12:20:32",Unidαd Absolutα,sticker omitido
61438,"19/05/22, 21:36:37",Marcelo,imagen omitida
61439,"19/05/22, 21:36:45",Marcelo,Por eso ya no tienes sueño en la chamba we


In [6]:
df.shape

(61441, 3)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61441 entries, 0 to 61440
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   t_fecha    61441 non-null  object
 1   v_usuario  61441 non-null  object
 2   t_mensaje  61441 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [8]:
df[df['t_fecha'].map(lambda x:not x[0].isdigit())]

,t_fecha,v_usuario,t_mensaje
583,-Claro que sí.,Claro que sí.,-Claro que sí.
1962,Tenía guardados 100,enía guardados 100,Tenía guardados 100
2113,Los hoteles están funcionando con el 10 o 15% ...,os hoteles están funcionando con el 10 o 15% d...,Los hoteles están funcionando con el 10 o 15% ...
2801,Sí ese fue el proyecto final que presentaste e...,í ese fue el proyecto final que presentaste en...,Sí ese fue el proyecto final que presentaste e...
2822,Y después de dos horas de estar pedaleando seg...,después de dos horas de estar pedaleando segu...,Y después de dos horas de estar pedaleando seg...
...,...,...,...
60336,Meses,eses,Meses
60337,Sin hacer ejercicio,in hacer ejercicio,Sin hacer ejercicio
60363,Neta es que sí quiero conocer a alguien brou,eta es que sí quiero conocer a alguien brou,Neta es que sí quiero conocer a alguien brou
60882,Empujo los frijoles jajajajaja,mpujo los frijoles jajajajaja,Empujo los frijoles jajajajaja


In [9]:
# Después de realizar la imputación
# Procedemos a eliminar estos registros
df = df[df['t_fecha'].map(lambda x:x[0].isdigit())]

In [10]:
df[df['t_fecha'].map(lambda x:not x[0].isdigit())]

,t_fecha,v_usuario,t_mensaje


## Antes de procesar el texto, se realizan las transformaciones pertinentes a los datos
- Formatear la fecha
- Extraer la hora en una columna aparte

In [11]:
df['t_fecha'] = pd.to_datetime(df['t_fecha'])

/home/sqpr14_/anaconda3/envs/dss/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [12]:
df['v_hora'] = df['t_fecha'].dt.hour

In [13]:
df = df[['t_fecha', 'v_hora', 't_mensaje', 'v_usuario']]

In [14]:
df['t_fecha'] = df['t_fecha'].dt.date

In [15]:
df

,t_fecha,v_hora,t_mensaje,v_usuario
0,2020-10-26,0,Los mensajes y las llamadas están cifrados de ...,Marcelo
1,2020-10-26,0,https://fb.watch/1m8FhbxovL/,Marcelo
2,2020-10-26,0,Buenas,Marcelo
3,2020-10-26,6,Wey,Unidαd Absolutα
4,2020-10-26,6,Está bien vergas este horario,Unidαd Absolutα
...,...,...,...,...
61436,2022-05-19,12,Va a salir con sida wey,Unidαd Absolutα
61437,2022-05-19,12,sticker omitido,Unidαd Absolutα
61438,2022-05-19,21,imagen omitida,Marcelo
61439,2022-05-19,21,Por eso ya no tienes sueño en la chamba we,Marcelo


In [16]:
df_m = pd.DataFrame(zip(list(df.columns), df.isnull().sum()), columns=['nombre_columna', 'total_missings'])

In [17]:
df_m

,nombre_columna,total_missings
0,t_fecha,0
1,v_hora,0
2,t_mensaje,0
3,v_usuario,0


In [18]:
def omitidos(x):
    x = x.lower()
    if('sticker omitido' in x or 'imagen omitida' in x or 'audio omitido'  in x or 'video omitido' in x):
        return True
    else:
        return False

In [19]:
df['t_mensaje'].map(omitidos).sum()

7177

In [20]:
total_multimedia = df[df['t_mensaje'].map(omitidos)]

In [21]:
total_multimedia

,t_fecha,v_hora,t_mensaje,v_usuario
34,2020-10-26,8,sticker omitido,Unidαd Absolutα
39,2020-10-26,8,sticker omitido,Unidαd Absolutα
41,2020-10-26,8,sticker omitido,Unidαd Absolutα
42,2020-10-26,8,sticker omitido,Unidαd Absolutα
43,2020-10-26,8,sticker omitido,Unidαd Absolutα
...,...,...,...,...
61431,2022-05-19,11,imagen omitida,Unidαd Absolutα
61435,2022-05-19,11,sticker omitido,Marcelo
61437,2022-05-19,12,sticker omitido,Unidαd Absolutα
61438,2022-05-19,21,imagen omitida,Marcelo


In [22]:
df.drop(total_multimedia.index, axis=0,inplace=True)
df = df.reset_index(drop=True)
df

,t_fecha,v_hora,t_mensaje,v_usuario
0,2020-10-26,0,Los mensajes y las llamadas están cifrados de ...,Marcelo
1,2020-10-26,0,https://fb.watch/1m8FhbxovL/,Marcelo
2,2020-10-26,0,Buenas,Marcelo
3,2020-10-26,6,Wey,Unidαd Absolutα
4,2020-10-26,6,Está bien vergas este horario,Unidαd Absolutα
...,...,...,...,...
53984,2022-05-19,11,Verga,Unidαd Absolutα
53985,2022-05-19,11,Verga,Marcelo
53986,2022-05-19,11,Va a salir así como con algo wey,Marcelo
53987,2022-05-19,12,Va a salir con sida wey,Unidαd Absolutα


## Limpieza de variables

### Limpieza de variable usuario

In [23]:
df['v_usuario'].unique()

array(['Marcelo', 'Unidαd Absolutα'], dtype=object)

In [24]:
df['v_usuario'] = df.v_usuario.map(lambda x:x.lower().replace(" ",'').replace('α','a'))

In [25]:
df

,t_fecha,v_hora,t_mensaje,v_usuario
0,2020-10-26,0,Los mensajes y las llamadas están cifrados de ...,marcelo
1,2020-10-26,0,https://fb.watch/1m8FhbxovL/,marcelo
2,2020-10-26,0,Buenas,marcelo
3,2020-10-26,6,Wey,unidadabsoluta
4,2020-10-26,6,Está bien vergas este horario,unidadabsoluta
...,...,...,...,...
53984,2022-05-19,11,Verga,unidadabsoluta
53985,2022-05-19,11,Verga,marcelo
53986,2022-05-19,11,Va a salir así como con algo wey,marcelo
53987,2022-05-19,12,Va a salir con sida wey,unidadabsoluta


### Limpieza de la variable mensaje

In [26]:
# Función para quitar los enlaces
enlaces = lambda x: True if ('http' in x.lower()) else False
df_aux = df[df['t_mensaje'].map(enlaces)]
df.drop(df_aux.index, axis=0, inplace=True)

#Funcion para quitar los mensajes de seguridad
seguridad = lambda x: True if('Los mensajes y las llamadas están cifrados de extremo a extremo.' in x 
                              or 'Cambió tu código de seguridad con' in x) else False
df_aux = df[df['t_mensaje'].map(seguridad)]
df.drop(df_aux.index, axis=0, inplace=True)
df = df.reset_index(drop=True)
del(df_aux)

In [27]:
df

,t_fecha,v_hora,t_mensaje,v_usuario
0,2020-10-26,0,Buenas,marcelo
1,2020-10-26,6,Wey,unidadabsoluta
2,2020-10-26,6,Está bien vergas este horario,unidadabsoluta
3,2020-10-26,7,No mames,marcelo
4,2020-10-26,7,Hermoso,marcelo
...,...,...,...,...
53665,2022-05-19,11,Verga,unidadabsoluta
53666,2022-05-19,11,Verga,marcelo
53667,2022-05-19,11,Va a salir así como con algo wey,marcelo
53668,2022-05-19,12,Va a salir con sida wey,unidadabsoluta


In [28]:
# Retirar cadenas de texto un sólo caracter que no sean emojis
def verificar_longitud(x):
    if len(x) == 1:
        if x in emoji.UNICODE_EMOJI:
            return True
        else:
            return False
    else:
        return True

In [29]:
df = df[df['t_mensaje'].map(verificar_longitud)]

In [30]:
df

,t_fecha,v_hora,t_mensaje,v_usuario
0,2020-10-26,0,Buenas,marcelo
1,2020-10-26,6,Wey,unidadabsoluta
2,2020-10-26,6,Está bien vergas este horario,unidadabsoluta
3,2020-10-26,7,No mames,marcelo
4,2020-10-26,7,Hermoso,marcelo
...,...,...,...,...
53665,2022-05-19,11,Verga,unidadabsoluta
53666,2022-05-19,11,Verga,marcelo
53667,2022-05-19,11,Va a salir así como con algo wey,marcelo
53668,2022-05-19,12,Va a salir con sida wey,unidadabsoluta


In [31]:
# Sólo por verificar cuantos mensajes son un sólo emoji
df['t_mensaje'].map(lambda x: True if x in emoji.UNICODE_EMOJI else False).sum()

2

In [32]:
def clean_text(text, pattern="[^a-zA-Z-0-9ñÑáéíóúüÁÉÍÓÚÜ\s]+"):
    cleaned_text = unicodedata.normalize('NFD', text).encode('utf-8', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().strip().split())
    return cleaned_text

In [33]:
df['t_mensaje_m'] = df['t_mensaje'].map(lambda x:clean_text(x).lower())

/home/sqpr14_/anaconda3/envs/dss/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [34]:
# Solo por curiosidad de saber la parte del día de mayor frecuencia
df.describe()

,v_hora
count,53527.000000
mean,13.775889
std,5.060785
min,0.000000
25%,10.000000
50%,14.000000
75%,17.000000
max,23.000000


## Ingeniería de variables

### STOP WORDS

In [35]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sqpr14_/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
s_w = nltk.corpus.stopwords.words('spanish')
len(s_w)

313

In [39]:
s_w

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [40]:
stop = lambda x: " ".join([i for i in x.split(' ') if i not in s_w])
df['t_mensaje_m'] = df['t_mensaje'].map(stop)

/home/sqpr14_/anaconda3/envs/dss/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [41]:
df

,t_fecha,v_hora,t_mensaje,v_usuario,t_mensaje_m
0,2020-10-26,0,Buenas,marcelo,Buenas
1,2020-10-26,6,Wey,unidadabsoluta,Wey
2,2020-10-26,6,Está bien vergas este horario,unidadabsoluta,Está bien vergas horario
3,2020-10-26,7,No mames,marcelo,No mames
4,2020-10-26,7,Hermoso,marcelo,Hermoso
...,...,...,...,...,...
53665,2022-05-19,11,Verga,unidadabsoluta,Verga
53666,2022-05-19,11,Verga,marcelo,Verga
53667,2022-05-19,11,Va a salir así como con algo wey,marcelo,Va salir así wey
53668,2022-05-19,12,Va a salir con sida wey,unidadabsoluta,Va salir sida wey


### HAPAXES

In [42]:
corpus = (' '.join(df['t_mensaje_m'].values)).split()

In [43]:
corpus

['Buenas',
 'Wey',
 'Está',
 'bien',
 'vergas',
 'horario',
 'No',
 'mames',
 'Hermoso',
 'Ayer',
 'hice',
 'chingo',
 'cosas',
 'casa',
 'Y',
 'dormí',
 'Si',
 'we',
 'Te',
 'rinde',
 'buen',
 'mañana',
 'Sí',
 'we',
 'En',
 'general',
 'Te',
 'rinde',
 'chido',
 'día',
 'Oye',
 'we',
 'Pudiste',
 'ponerle',
 'cuenta',
 'banco',
 'iPhone?',
 'Sin',
 'pedos',
 'Neta?',
 'Cómo?',
 'Nadamas',
 'Se',
 'quiero',
 'poner',
 'iPod',
 'Puse',
 'cuenta',
 'Google',
 'Y',
 'pidió',
 'iniciar',
 'sesión',
 'En',
 'entorno',
 'colaborativo',
 'Y',
 'we',
 'No',
 'mames',
 'No',
 'encuentro',
 'dónde',
 'Alv',
 'Nadamas',
 'agrega',
 'cuenta',
 'gugul',
 'Y',
 'we',
 'Solito',
 'sale',
 'Pero',
 'parte',
 'De',
 'ajustes?',
 'En',
 'ninguno',
 'wey',
 'Te',
 'sale',
 'solo',
 'agregar',
 'cuenta',
 'gugul',
 'cabrona',
 'madre',
 'Se',
 'eliminó',
 'mensaje.',
 'Agregas',
 'cuenta',
 'La',
 'banco',
 'Y',
 'solito',
 'manda',
 'iniciar',
 'sesión',
 'Del',
 'banco',
 'Jaja',
 'hiciste',
 'eso?',
 

In [44]:
fdist = nltk.FreqDist(corpus)

In [45]:
fdist

FreqDist({'No': 4248, 'we': 3974, 'wey': 3879, 'Y': 3599, 'mames': 2464, 'Pero': 2415, 'Sí': 1362, 'Pues': 1301, 'Wey': 1202, 'Ya': 1111, ...})

In [46]:
pd.DataFrame(fdist.items()).sort_values(by=[1])

,0,1
22922,Jajajajajajajajj,1
10947,animaciones,1
10948,grababa,1
18196,Grandote,1
10950,Kermoso,1
...,...,...
7,mames,2464
14,Y,3599
61,wey,3879
17,we,3974


In [47]:
list_hapaxes=fdist.hapaxes()

In [48]:
len(list_hapaxes)

13909

In [49]:
list_hapaxes

['agrega',
 'Solito',
 'ajustes?',
 'Agregas',
 'Uhhh',
 'boxlunch',
 'trigo',
 'cereales',
 'Frases',
 'sanandres',
 'Fierro',
 'mataba',
 'estàn',
 'adios',
 'jungla?',
 '¿Esos',
 '"esos',
 'argentinos"',
 '“Tremendos',
 'pajudos',
 'pija”',
 'Efectivamente,',
 'traba',
 'reproducinedo',
 'bluettoth',
 'trabarse',
 'atora',
 'Hahhhahaahhahaahhaahahahah',
 'hahahahahahahaha',
 'hinche',
 'dijiste??',
 'Visto,',
 'lora',
 'mamaditas',
 'Julia',
 'entonces,',
 'Media',
 'amonos',
 'misma?',
 'Vin',
 'Petrol',
 'reír?',
 'Turbogei',
 'medios',
 'tetos',
 'guardaste',
 'crah',
 'Crash',
 'Descárgalo',
 'id',
 'iMessage',
 'finalmente',
 'Norma?',
 'presentaciones',
 'paco',
 'leyva',
 'directivos',
 'Inyinirin',
 'CIb',
 'gemidos',
 'PENDEJOS',
 '“Se',
 'emojis',
 'iPhone”',
 '“Excelente',
 'idea”',
 'JJjaja',
 'boomers?',
 'concuerdan',
 'suéter',
 'Spock',
 'partiendo',
 'Jacqui',
 'mateos',
 'sujetarte',
 '“esta”',
 'sujeto,',
 'comedia?',
 'latieron',
 'maaameees',
 'monki',
 'monky',

In [50]:
df['t_mensaje_m']=df['t_mensaje_m'].map(lambda t :' '.join([x for x in t.split(" ") if x not in list_hapaxes]))

/home/sqpr14_/anaconda3/envs/dss/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [51]:
df

,t_fecha,v_hora,t_mensaje,v_usuario,t_mensaje_m
0,2020-10-26,0,Buenas,marcelo,Buenas
1,2020-10-26,6,Wey,unidadabsoluta,Wey
2,2020-10-26,6,Está bien vergas este horario,unidadabsoluta,Está bien vergas horario
3,2020-10-26,7,No mames,marcelo,No mames
4,2020-10-26,7,Hermoso,marcelo,Hermoso
...,...,...,...,...,...
53665,2022-05-19,11,Verga,unidadabsoluta,Verga
53666,2022-05-19,11,Verga,marcelo,Verga
53667,2022-05-19,11,Va a salir así como con algo wey,marcelo,Va salir así wey
53668,2022-05-19,12,Va a salir con sida wey,unidadabsoluta,Va salir sida wey


### Tokenización

In [52]:
df['t_mensaje_m'] = df['t_mensaje_m'].map(lambda x:x.split())

/home/sqpr14_/anaconda3/envs/dss/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [53]:
df

,t_fecha,v_hora,t_mensaje,v_usuario,t_mensaje_m
0,2020-10-26,0,Buenas,marcelo,[Buenas]
1,2020-10-26,6,Wey,unidadabsoluta,[Wey]
2,2020-10-26,6,Está bien vergas este horario,unidadabsoluta,"[Está, bien, vergas, horario]"
3,2020-10-26,7,No mames,marcelo,"[No, mames]"
4,2020-10-26,7,Hermoso,marcelo,[Hermoso]
...,...,...,...,...,...
53665,2022-05-19,11,Verga,unidadabsoluta,[Verga]
53666,2022-05-19,11,Verga,marcelo,[Verga]
53667,2022-05-19,11,Va a salir así como con algo wey,marcelo,"[Va, salir, así, wey]"
53668,2022-05-19,12,Va a salir con sida wey,unidadabsoluta,"[Va, salir, sida, wey]"


### Estematización en español

In [54]:
from nltk.stem import SnowballStemmer
estem_esp = SnowballStemmer('spanish')
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/sqpr14_/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [55]:
s = 'Por eso ya no tienes sueño en la chamba we'
print(estem_esp.stem(s))

por eso ya no tienes sueño en la chamba w


In [56]:
def estematiza(x):
    return [estem_esp.stem(i) for i in x]

In [57]:
df['t_mensaje_m'] = df['t_mensaje_m'].map(estematiza)

/home/sqpr14_/anaconda3/envs/dss/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [58]:
df

,t_fecha,v_hora,t_mensaje,v_usuario,t_mensaje_m
0,2020-10-26,0,Buenas,marcelo,[buen]
1,2020-10-26,6,Wey,unidadabsoluta,[wey]
2,2020-10-26,6,Está bien vergas este horario,unidadabsoluta,"[esta, bien, verg, horari]"
3,2020-10-26,7,No mames,marcelo,"[no, mam]"
4,2020-10-26,7,Hermoso,marcelo,[hermos]
...,...,...,...,...,...
53665,2022-05-19,11,Verga,unidadabsoluta,[verg]
53666,2022-05-19,11,Verga,marcelo,[verg]
53667,2022-05-19,11,Va a salir así como con algo wey,marcelo,"[va, sal, asi, wey]"
53668,2022-05-19,12,Va a salir con sida wey,unidadabsoluta,"[va, sal, sid, wey]"


In [59]:
#NUMERO DE PALABRAS
df['c_numero_palabras'] = df["t_mensaje_m"].apply(lambda x: len(x))

/home/sqpr14_/anaconda3/envs/dss/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [60]:
#NUMERO DE LETRAS POR SENTENCIA
df['c_numero_letras'] =df["t_mensaje_m"].map(lambda x: sum(len(word) for word in x))

In [61]:
#LONGITUD PROMEDIO DE PALABRAS
df['c_long_pro_pala'] = df['c_numero_letras'] / df['c_numero_palabras']

/home/sqpr14_/anaconda3/envs/dss/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [62]:
df

,t_fecha,v_hora,t_mensaje,v_usuario,t_mensaje_m,c_numero_palabras,c_numero_letras,c_long_pro_pala
0,2020-10-26,0,Buenas,marcelo,[buen],1,4,4.00
1,2020-10-26,6,Wey,unidadabsoluta,[wey],1,3,3.00
2,2020-10-26,6,Está bien vergas este horario,unidadabsoluta,"[esta, bien, verg, horari]",4,18,4.50
3,2020-10-26,7,No mames,marcelo,"[no, mam]",2,5,2.50
4,2020-10-26,7,Hermoso,marcelo,[hermos],1,6,6.00
...,...,...,...,...,...,...,...,...
53665,2022-05-19,11,Verga,unidadabsoluta,[verg],1,4,4.00
53666,2022-05-19,11,Verga,marcelo,[verg],1,4,4.00
53667,2022-05-19,11,Va a salir así como con algo wey,marcelo,"[va, sal, asi, wey]",4,11,2.75
53668,2022-05-19,12,Va a salir con sida wey,unidadabsoluta,"[va, sal, sid, wey]",4,11,2.75


### TF-IDF VECTORIZER

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [64]:
X_train,X_test=train_test_split(df,test_size=0.20,random_state=42)

In [65]:
vect = TfidfVectorizer()
vect.fit(X_train["t_mensaje"])
array_train= vect.transform(X_train["t_mensaje"])
array_test= vect.transform(X_test["t_mensaje"])

In [66]:
aux_train=pd.DataFrame(array_train.toarray(),columns=vect.get_feature_names())
aux_test=pd.DataFrame(array_train.toarray(),columns=vect.get_feature_names())

In [67]:
aux_train

,00,000,00s,01,02,0km,10,100,1000,10000,...,últimos,últmas,únete,única,únicas,único,únicos,úrsula,útil,útiles
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
X_train

,t_fecha,v_hora,t_mensaje,v_usuario,t_mensaje_m,c_numero_palabras,c_numero_letras,c_long_pro_pala
8375,2021-01-20,13,Wey,unidadabsoluta,[wey],1,3,3.000000
51242,2022-01-04,20,En mi caso está de wevos eso,unidadabsoluta,"[en, cas, wev]",3,8,2.666667
7577,2021-01-18,13,Imagínate yo,unidadabsoluta,[imaginat],1,8,8.000000
29859,2021-12-07,9,No te va a dar neumonía,unidadabsoluta,"[no, va, dar, neumon]",4,13,3.250000
43004,2021-05-11,14,Al de seguridad wey,marcelo,"[al, segur, wey]",3,10,3.333333
...,...,...,...,...,...,...,...,...
11310,2021-07-02,17,No te hagas pendejo,unidadabsoluta,"[no, hag, pendej]",3,11,3.666667
44855,2021-12-21,9,Del primer Gaia,unidadabsoluta,"[del, prim, gai]",3,10,3.333333
38264,2021-09-21,9,Sí we,unidadabsoluta,"[si, we]",2,4,2.000000
864,2020-03-11,18,Tú me la quieres mamar,marcelo,"[tu, quier, mam]",3,10,3.333333


In [69]:
X_train=pd.concat([X_train.reset_index(drop=True),aux_train.reset_index(drop=True)],axis=1)
X_test=pd.concat([X_test.reset_index(drop=True),aux_test.reset_index(drop=True)],axis=1)

In [72]:
df_text = X_train.copy()

In [73]:
del(df)